In [ ]:
import mlflow
import pandas as pd
mlflow.set_tracking_uri("https://mlflow-g4k-serving-474827717259.europe-west3.run.app/")
mlflow.set_experiment("ExpRAG")
df = mlflow.search_runs(
    experiment_names=["ExpRAG"],
)

In [ ]:
df

In [ ]:
# Check columns
print("Columns:", df.columns.tolist())
print("\nShape:", df.shape)
print("\nFirst few rows:")
df.head()

In [ ]:
# Filter for gemma3 model
df_gemma3 = df[df['params.model_model_name_short'] == 'gemma3'].copy()
print(f"Total gemma3 runs: {len(df_gemma3)}")
print(f"\nUnique rag_methods: {df_gemma3['params.rag_method'].unique()}")
print(f"\nUnique dataset_subsets: {df_gemma3['params.dataset_subset'].unique()}")

In [ ]:
# Check what models are available
print("Available model_model_name_short values:")
print(df['params.model_model_name_short'].value_counts())
print("\n\nCheck if there's a similar column:")
model_cols = [col for col in df.columns if 'model' in col.lower() and 'name' in col.lower()]
print("\nModel-related columns:", model_cols)

In [ ]:
# Filter for gemma3 model (using gemma3-27b-it)
df_gemma3 = df[df['params.model_model_name_short'] == 'gemma3-27b-it'].copy()
print(f"Total gemma3 runs: {len(df_gemma3)}")
print(f"\nUnique rag_methods: {sorted(df_gemma3['params.rag_method'].unique())}")
print(f"\nUnique dataset_subsets: {sorted(df_gemma3['params.dataset_subset'].unique())}")

# Check for NaN values in key columns
print(f"\nMissing values in metrics.mrr: {df_gemma3['metrics.mrr'].isna().sum()}")
print(f"Missing values in metrics.f1: {df_gemma3['metrics.f1'].isna().sum()}")

In [ ]:
# Check which F1 metric has data
print("Available metrics columns:")
metric_cols = [col for col in df_gemma3.columns if col.startswith('metrics.')]
for col in metric_cols:
    non_na = df_gemma3[col].notna().sum()
    if non_na > 0:
        print(f"{col}: {non_na} non-null values")
        
# Check sample values
print("\n\nSample MRR values:", df_gemma3['metrics.mrr'].head(5).tolist())
print("Sample F1squad values:", df_gemma3['metrics.f1squad'].head(5).tolist())

In [ ]:
# Remove rows with missing MRR or F1squad values
df_gemma3_clean = df_gemma3.dropna(subset=['metrics.mrr', 'metrics.f1squad'])

# Scale metrics to 0-100 range
df_gemma3_clean['mrr_scaled'] = (df_gemma3_clean['metrics.mrr'] * 100).round(1)
df_gemma3_clean['f1_scaled'] = (df_gemma3_clean['metrics.f1squad'] * 100).round(1)

# Create aggregated data grouped by rag_method and dataset_subset
agg_data = df_gemma3_clean.groupby(['params.rag_method', 'params.dataset_subset']).agg({
    'mrr_scaled': 'mean',
    'f1_scaled': 'mean'
}).round(1).reset_index()

print("Aggregated data:")
print(agg_data.head(10))
print(f"\nTotal rows: {len(agg_data)}")

In [ ]:
# Modified filtering: Include rows with both MRR and F1squad values,
# OR no_context runs that have F1squad values (even if MRR is missing)
mask_complete = df_gemma3['metrics.mrr'].notna() & df_gemma3['metrics.f1squad'].notna()
mask_no_context_with_f1 = (df_gemma3['params.rag_method'] == 'no_context') & df_gemma3['metrics.f1squad'].notna()

df_gemma3_clean = df_gemma3[mask_complete | mask_no_context_with_f1].copy()

print(f"Total rows included: {len(df_gemma3_clean)}")
print(f"  - Rows with both MRR and F1squad: {mask_complete.sum()}")
print(f"  - no_context rows with F1squad (may lack MRR): {mask_no_context_with_f1.sum()}")

# Scale metrics to 0-100 range
# For MRR, handle NaN values (they'll stay as NaN)
df_gemma3_clean['mrr_scaled'] = (df_gemma3_clean['metrics.mrr'] * 100).round(1)
df_gemma3_clean['f1_scaled'] = (df_gemma3_clean['metrics.f1squad'] * 100).round(1)

# Create aggregated data grouped by rag_method and dataset_subset
agg_data = df_gemma3_clean.groupby(['params.rag_method', 'params.dataset_subset']).agg({
    'mrr_scaled': 'mean',
    'f1_scaled': 'mean'
}).round(1).reset_index()

# Pivot to create matrix: rows = rag_method, columns = dataset_subset
# We'll create separate pivots for MRR and F1
pivot_mrr = agg_data.pivot(index='params.rag_method', columns='params.dataset_subset', values='mrr_scaled')
pivot_f1 = agg_data.pivot(index='params.rag_method', columns='params.dataset_subset', values='f1_scaled')

# Combine the three DoQA datasets into one column by averaging
doqa_datasets = ['doqa_cooking', 'doqa_movies', 'doqa_travel']
doqa_cols_present = [col for col in doqa_datasets if col in pivot_mrr.columns]



In [ ]:
def create_latex_table(pivot_mrr, pivot_f1):
    """Create a LaTeX table with MRR and F1 scores"""
    
    # Get sorted indices and columns
    rag_methods = sorted(pivot_mrr.index)
    datasets = sorted(pivot_mrr.columns)
    
    # Start building the LaTeX table
    num_cols = len(datasets)
    # Each dataset has 2 columns (MRR and F1)
    col_format = 'l|' + ''.join(['cc|' for _ in range(num_cols)])
    
    latex = []
    latex.append(r'\begin{table}[h]')
    latex.append(r'\centering')
    latex.append(r'\begin{tabular}{' + col_format + '}')
    latex.append(r'\hline')
    
    # Header row 1: Dataset names (spanning 2 columns each)
    header1 = r'RAG Method'
    for dataset in datasets:
        header1 += r' & \multicolumn{2}{c|}{' + dataset.replace('_', r'\_') + '}'
    header1 += r' \\'
    latex.append(header1)
    
    # Header row 2: MRR and F1 labels
    header2 = ''
    for _ in datasets:
        header2 += r' & MRR & F1'
    header2 += r' \\'
    latex.append(header2)
    latex.append(r'\hline')
    
    # Data rows
    for rag_method in rag_methods:
        row = rag_method.replace('_', r'\_')
        for dataset in datasets:
            mrr_val = pivot_mrr.loc[rag_method, dataset] if dataset in pivot_mrr.columns and rag_method in pivot_mrr.index else None
            f1_val = pivot_f1.loc[rag_method, dataset] if dataset in pivot_f1.columns and rag_method in pivot_f1.index else None
            
            mrr_str = f'{mrr_val:.1f}' if pd.notna(mrr_val) else '-'
            f1_str = f'{f1_val:.1f}' if pd.notna(f1_val) else '-'
            
            row += f' & {mrr_str} & {f1_str}'
        row += r' \\'
        latex.append(row)
    
    latex.append(r'\hline')
    latex.append(r'\end{tabular}')
    latex.append(r'\caption{MRR and F1 scores for Gemma3-27B-IT across different RAG methods and datasets}')
    latex.append(r'\label{tab:gemma3_results}')
    latex.append(r'\end{table}')
    
    return '\n'.join(latex)

# Generate the LaTeX table
latex_table = create_latex_table(pivot_mrr, pivot_f1)
print(latex_table)

In [ ]:
# Save the LaTeX table to a file
output_file = '/ltstorage/home/strich/exp-rag/src/gemma3_results_table.tex'
with open(output_file, 'w') as f:
    f.write(latex_table)
print(f"LaTeX table saved to: {output_file}")

In [ ]:
# Find all missing combinations
all_rag_methods = sorted(pivot_mrr.index)
all_datasets = sorted(pivot_mrr.columns)

missing_combinations = []

for rag_method in all_rag_methods:
    for dataset in all_datasets:
        mrr_val = pivot_mrr.loc[rag_method, dataset]
        f1_val = pivot_f1.loc[rag_method, dataset]
        
        # Check if data is missing (NaN)
        if pd.isna(mrr_val) or pd.isna(f1_val):
            missing_combinations.append({
                'RAG Method': rag_method,
                'Dataset': dataset,
                'MRR Missing': pd.isna(mrr_val),
                'F1 Missing': pd.isna(f1_val)
            })

# Create a DataFrame for better visualization
missing_df = pd.DataFrame(missing_combinations)

print(f"Total missing combinations: {len(missing_combinations)} out of {len(all_rag_methods) * len(all_datasets)} possible combinations")
print(f"Coverage: {((len(all_rag_methods) * len(all_datasets) - len(missing_combinations)) / (len(all_rag_methods) * len(all_datasets)) * 100):.1f}%")
print("\nMissing combinations:")
print(missing_df.to_string(index=False))

In [ ]:
# Check what RAG methods are available for gemma3 in the entire dataset
print("All RAG methods available for gemma3-27b-it:")
print(sorted(df_gemma3['params.rag_method'].unique()))
print(f"\nTotal unique RAG methods: {df_gemma3['params.rag_method'].nunique()}")

# Count runs per RAG method
print("\nRuns per RAG method:")
print(df_gemma3['params.rag_method'].value_counts().sort_index())

In [ ]:
# Check if the other methods exist in the full dataset (any model)
requested_methods = ['base', 'hybrid', 'hyde_reranker', 'hyde', 'known_context', 
                     'no_context', 'query_rewriting', 'reranker', 
                     'summarization_context', 'summarization']

all_methods_in_df = df['params.rag_method'].unique()
print("All RAG methods in entire dataset:")
print(sorted([m for m in all_methods_in_df if pd.notna(m)]))

print("\n\nChecking requested methods:")
for method in requested_methods:
    count = (df['params.rag_method'] == method).sum()
    gemma3_count = (df_gemma3['params.rag_method'] == method).sum()
    print(f"{method:25} - Total: {count:4}, Gemma3: {gemma3_count:4}")

In [ ]:
# Create a complete table with all requested methods
requested_methods = ['base', 'hybrid_bm25', 'hyde_reranker', 'hyde', 'known_context', 
                     'no_context', 'query_rewriting', 'reranker', 
                     'summarization_context', 'summarization']

# Note: Using 'hybrid_bm25' instead of 'hybrid' as that's what exists in the data

# Get all datasets from the current data
all_datasets_available = sorted(df_gemma3['params.dataset_subset'].unique())

print("Requested methods for complete table:")
print(requested_methods)
print(f"\nAvailable datasets: {all_datasets_available}")
print(f"\nGenerating complete coverage matrix...")

# Create a coverage matrix
coverage_data = []
for method in requested_methods:
    for dataset in all_datasets_available:
        matching_runs = df_gemma3_clean[
            (df_gemma3_clean['params.rag_method'] == method) & 
            (df_gemma3_clean['params.dataset_subset'] == dataset)
        ]
        
        has_data = len(matching_runs) > 0
        coverage_data.append({
            'Method': method,
            'Dataset': dataset,
            'Has Data': 'Yes' if has_data else 'No',
            'Num Runs': len(matching_runs)
        })

coverage_df = pd.DataFrame(coverage_data)
coverage_pivot = coverage_df.pivot(index='Method', columns='Dataset', values='Has Data')

print("\nData Coverage Matrix:")
print(coverage_pivot)

In [ ]:
# Create complete pivot tables with all requested methods
requested_methods_sorted = ['base', 'hybrid_bm25', 'hyde', 'hyde_reranker', 'known_context', 
                            'no_context', 'query_rewriting', 'reranker', 
                            'summarization', 'summarization_context']

all_datasets_sorted = sorted(all_datasets_available)

# Create pivots with reindex to include all requested methods
pivot_mrr_complete = agg_data.pivot(index='params.rag_method', columns='params.dataset_subset', values='mrr_scaled')
pivot_f1_complete = agg_data.pivot(index='params.rag_method', columns='params.dataset_subset', values='f1_scaled')

# Reindex to include all requested methods and datasets
pivot_mrr_complete = pivot_mrr_complete.reindex(index=requested_methods_sorted, columns=all_datasets_sorted)
pivot_f1_complete = pivot_f1_complete.reindex(index=requested_methods_sorted, columns=all_datasets_sorted)

# Combine the three DoQA datasets into one column by averaging
doqa_datasets = ['doqa_cooking', 'doqa_movies', 'doqa_travel']
doqa_cols_present = [col for col in doqa_datasets if col in pivot_mrr_complete.columns]

if doqa_cols_present:
    # Calculate mean for MRR and F1
    pivot_mrr_complete['doqa_mean'] = pivot_mrr_complete[doqa_cols_present].mean(axis=1)
    pivot_f1_complete['doqa_mean'] = pivot_f1_complete[doqa_cols_present].mean(axis=1)



In [ ]:
# Generate the complete LaTeX table
latex_table_complete = create_latex_table(pivot_mrr_complete, pivot_f1_complete)
print(latex_table_complete)

# Also save it
output_file_complete = '/ltstorage/home/strich/exp-rag/src/gemma3_results_table_complete.tex'
with open(output_file_complete, 'w') as f:
    f.write(latex_table_complete)
print(f"\n\nComplete LaTeX table saved to: {output_file_complete}")

In [ ]:
# Summary of missing data
total_combinations = len(requested_methods_sorted) * len(all_datasets_sorted)
missing_count = 0

print("=" * 80)
print("SUMMARY: Missing Data for Gemma3-27B-IT")
print("=" * 80)

for method in requested_methods_sorted:
    method_missing = []
    for dataset in all_datasets_sorted:
        if pd.isna(pivot_mrr_complete.loc[method, dataset]):
            method_missing.append(dataset)
            missing_count += 1
    
    if method_missing:
        print(f"\n{method:25} - Missing {len(method_missing)}/10 datasets:")
        if len(method_missing) == len(all_datasets_sorted):
            print(f"  → NO DATA AVAILABLE FOR THIS METHOD")
        else:
            print(f"  → Missing: {', '.join(method_missing)}")
    else:
        print(f"\n{method:25} - COMPLETE (all 10 datasets)")

print("\n" + "=" * 80)
print(f"Total coverage: {total_combinations - missing_count}/{total_combinations} ({(total_combinations - missing_count)/total_combinations*100:.1f}%)")
print(f"Missing combinations: {missing_count}")
print("=" * 80)

In [ ]:
# Check for no_context runs with F1 values (even if status is failed or MRR is missing)
no_context_runs = df_gemma3[df_gemma3['params.rag_method'] == 'no_context'].copy()

print(f"Total no_context runs: {len(no_context_runs)}")
print(f"\nRuns with F1squad values: {no_context_runs['metrics.f1squad'].notna().sum()}")
print(f"Runs with MRR values: {no_context_runs['metrics.mrr'].notna().sum()}")

if 'status' in no_context_runs.columns:
    print(f"\nStatus distribution:")
    print(no_context_runs['status'].value_counts())
    
    print(f"\nFailed runs with F1squad values: {no_context_runs[(no_context_runs['status'] == 'FAILED') & (no_context_runs['metrics.f1squad'].notna())].shape[0]}")

# Check what's currently excluded from df_gemma3_clean
currently_excluded = df_gemma3[
    (df_gemma3['params.rag_method'] == 'no_context') & 
    (~df_gemma3.index.isin(df_gemma3_clean.index))
]

print(f"\nCurrently excluded no_context runs: {len(currently_excluded)}")
print(f"Of those, how many have F1squad values: {currently_excluded['metrics.f1squad'].notna().sum()}")

if len(currently_excluded) > 0 and currently_excluded['metrics.f1squad'].notna().sum() > 0:
    print("\nExcluded no_context runs with F1 values:")
    print(currently_excluded[currently_excluded['metrics.f1squad'].notna()][
        ['params.dataset_subset', 'metrics.f1squad', 'metrics.mrr', 'status']
    ].to_string())